# LeafScans WorkBook 3
* Create Train-Test Loop for ML predictions
    * Partial Least Squares Regression
    * Random Forrest Regression
    * Support Vector Machine Regression
* For each ML algorithm and for each target data set (leaf nutes or leaf gas exchange data)
    * Split into 5 random folds with respect to genotype (avoid data leakage)
    * Use grid search and recursive feature elimination (not for svm) to optimize model hyperparams in training set
    * Apply and record R2 on testing set and record score


In [1]:
# Import modules
# Fundamental data manipulationa nd visualization packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


# machine learning and data preprocessing tools from sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import GridSearchCV


# set seed for torch and other ML functions
np.random.seed(42)

In [2]:
df_nutes = pd.read_csv(r'../Data/NuteScansAll_merged.csv')
df_gasex = pd.read_csv(r'../Data/GasExScansAll_Merged.csv')
# Calculate NDVI based on lit
df_nutes['ndvi'] = (df_nutes['800'] - df_nutes['651'])/(df_nutes['800'] + df_nutes['651'])
df_nutes_R2 = df_nutes[df_nutes['DevStage'] == 'R2'].reset_index()
# Wavelengths list usefule for graphing and looping
wavelengths = df_gasex.columns[20:355].astype(str)

In [3]:
def pls_variable_selection(X, y, max_comp):
    # Create an empty array to store mse values
    mse = np.zeros((max_comp,X.shape[1]))
    # Outer loop optimizes num of components
    for i in range(max_comp):
        #print(X.shape, y.shape)
        #print(i)
        pls1 = PLSRegression(n_components=i+1)
        pls1.fit(X, y)
        sorted_ind = np.argsort(np.abs(pls1.coef_[0]))
        #print(sorted_ind)
        Xc = X[:,sorted_ind]
        # Inner loop optimizes input features
        for j in range(Xc.shape[1]-(i+1)):
            pls2 = PLSRegression(n_components=i+1)
            pls2.fit(Xc[:, j:], y)
            p = cross_val_score(pls2, Xc[:, j:], y, scoring = 'neg_mean_squared_error')
            p = np.mean(np.abs(p))
            mse[i,j] = p
        comp = 100*(i+1)/(max_comp)
    #Calculate and print the position of minimum in MSE
    mseminx,mseminy = np.where(mse==np.min(mse[np.nonzero(mse)]))
    rpd_cv = y.std()/np.sqrt(mse[mseminx,mseminy][0])
    print("Optimised number of PLS components: ", mseminx[0]+1)
    print("Wavelengths to be discarded ",mseminy[0])
    print('Optimised MSEP ', mse[mseminx,mseminy][0])
    print('RPD CV', rpd_cv)
    # Build model with optimisez params for output
    Opt_components = mseminx[0]+1
    pls = PLSRegression(n_components=Opt_components)
    pls.fit(X, y)
    sorted_ind = np.argsort(np.abs(pls.coef_[0]))
    Xc = X[:, sorted_ind]
    wavD = mseminy[0]
    # Slice of sorted X matrix that only contains kept wavelengths
    Opt_x = Xc[:, wavD:]
    pls.fit(Opt_x, y)
    return (pls, Opt_components, wavD, sorted_ind)

In [4]:
print(df_gasex.columns[3:8])
targets = list(df_gasex.columns[3:8])
gkf = GroupKFold(n_splits=5)

Index(['E', 'A', 'Ci', 'gsw', 'WUE'], dtype='object')


In [5]:
scores = {}
for target in targets:
    y = df_gasex[target].values
    x = df_gasex[wavelengths].values
    groups = np.array(df_gasex['Name'])
    scores[target] = []
    i = 0

    for train_ind, test_ind in gkf.split(X = x, y = y, groups = groups):
        #print(df_gasex.loc[test_ind, 'Name'])
        pls, Opt_components, wavD, sorted_ind = pls_variable_selection(x[train_ind], y[train_ind], 9)
        #sns.histplot(x = y[test_ind].flatten(), hue = df_gasex.loc[test_ind, 'Ntreatment'])
        #plt.title(target + '-CV{}'.format(i))
        #plt.show()
        i += 1
        x_test = x[test_ind]
        x_test = x_test[:, sorted_ind]
        x_test = x_test[:, wavD:]
        y_predict = pls.predict(x_test)
        r2 = r2_score(y[test_ind], y_predict)
        print('External Validation Goodness-of-Fit', r2)
        scores[target].append(r2)

df_scores_GasExPLSR = pd.DataFrame(scores)
# df_scores_GasExPLSR.to_csv('Scores_GasExPLSR.csv', index = False)

Optimised number of PLS components:  6
Wavelengths to be discarded  328
Optimised MSEP  7.414408367227667e-07
RPD CV 1.146807949776514
External Validation Goodness-of-Fit 0.304150732527382
Optimised number of PLS components:  9
Wavelengths to be discarded  285
Optimised MSEP  7.724284391788038e-07
RPD CV 1.2083254839455317
External Validation Goodness-of-Fit 0.26135255928161993
Optimised number of PLS components:  9
Wavelengths to be discarded  285
Optimised MSEP  8.127483078265106e-07
RPD CV 1.1943125742017064
External Validation Goodness-of-Fit 0.104097445379565
Optimised number of PLS components:  9
Wavelengths to be discarded  312
Optimised MSEP  7.523717530374632e-07
RPD CV 1.222830309107544
External Validation Goodness-of-Fit -0.20486216824259373
Optimised number of PLS components:  9
Wavelengths to be discarded  301
Optimised MSEP  7.72355298804157e-07
RPD CV 1.1912239146149006
External Validation Goodness-of-Fit 0.13395211323472667
Optimised number of PLS components:  9
Wavelen

In [13]:
df_scores_GasExPLSR.to_clipboard()

In [6]:
# Same for Nutes
gkf = GroupKFold(n_splits=5)
print(df_nutes.columns[18:30])
targets = list(df_nutes.columns[18:30])

Index(['N', 'P', 'K', 'Ca', 'Mg', 'Na', 'S', 'Zn', 'Mn', 'Fe', 'Cu', 'B'], dtype='object')


In [7]:
scores = {}
for target in targets:
    y = df_nutes_R2[target].values
    x = df_nutes_R2[wavelengths].values
    groups = np.array(df_nutes_R2['Hybrid'])
    scores[target] = []
    i = 0

    for train_ind, test_ind in gkf.split(X = x, y = y, groups = groups):
        #print(df_gasex.loc[test_ind, 'Name'])
        pls, Opt_components, wavD, sorted_ind = pls_variable_selection(x[train_ind], y[train_ind], 9)
        #sns.histplot(x = y[test_ind].flatten(), hue = df_gasex.loc[test_ind, 'Ntreatment'])
        #plt.title(target + '-CV{}'.format(i))
        #plt.show()
        i += 1
        x_test = x[test_ind]
        x_test = x_test[:, sorted_ind]
        x_test = x_test[:, wavD:]
        y_predict = pls.predict(x_test)
        r2 = r2_score(y[test_ind], y_predict)
        print('External Validation Goodness-of-Fit', r2)
        scores[target].append(r2)

df_scores_NutesPLSR = pd.DataFrame(scores)
df_scores_NutesPLSR.to_csv('Scores_NutesPLSR.csv', index = False)

Optimised number of PLS components:  4
Wavelengths to be discarded  73
Optimised MSEP  0.07518247341257248
RPD CV 1.7706935371599193
External Validation Goodness-of-Fit 0.6405816046081032
Optimised number of PLS components:  9
Wavelengths to be discarded  314
Optimised MSEP  0.06538286550575975
RPD CV 1.9144271523870053
External Validation Goodness-of-Fit 0.6786202591289461
Optimised number of PLS components:  9
Wavelengths to be discarded  283
Optimised MSEP  0.06861184003679607
RPD CV 1.80032952776429
External Validation Goodness-of-Fit 0.6757323891746749
Optimised number of PLS components:  9
Wavelengths to be discarded  315
Optimised MSEP  0.0630756565596303
RPD CV 1.9309006632986156
External Validation Goodness-of-Fit 0.5758553833166442
Optimised number of PLS components:  4
Wavelengths to be discarded  130
Optimised MSEP  0.08572578992021525
RPD CV 1.6493027551219037
External Validation Goodness-of-Fit 0.7490779187514431
Optimised number of PLS components:  9
Wavelengths to be di

In [12]:
df_scores_NutesPLSR.to_clipboard()

In [5]:
from sklearn.model_selection import GridSearchCV
def RF_variable_selection(x, y):
    mse = np.zeros(x.shape[1])
    param_grid = {'n_estimators': [50, 100, 150]}
    rf1 = RandomForestRegressor()
    grid_search = GridSearchCV(estimator = rf1, param_grid=param_grid, cv = 3)
    grid_search.fit(x, y)
    best_rf = grid_search.best_estimator_
    # Sort spectra according to ascending feature_importances
    sorted_ind = np.argsort(best_rf.feature_importances_)
    x_sort = x[:,sorted_ind]
    # discard one wavelength at a time calculate and store mse
    for w in range(0, x_sort.shape[1], 5):
        best_rf.fit(x_sort[:, w:], y)
        predictions = best_rf.predict(x_sort[:, w:])
        p = cross_val_score(best_rf, x_sort[:, w:], y, scoring = 'neg_mean_squared_error')
        p = np.mean(np.abs(p))
        mse[w] = p
    mseminx = np.where(mse==np.min(mse[np.nonzero(mse)]))
    rpd_cv = y.std()/np.sqrt(mse[mseminx][0])
    print('Optimised MSEP ', mse[mseminx][0])
    print("Wavelengths to be discarded:", mseminx[0][0])
    # return the optimised model
    wavD = mseminx[0][0]
    best_rf.fit(x_sort[:, wavD:], y)
    predictions = best_rf.predict(x_sort[:, wavD:])
    print(r2_score(y, predictions))
    return best_rf, mse, wavD, sorted_ind

In [6]:
## GASEX RF-R ##

print(df_gasex.columns[3:8])
targets = list(df_gasex.columns[3:8])
gkf = GroupKFold(n_splits=5)
scores = {}
for target in targets:
    y = df_gasex[target].values
    x = df_gasex[wavelengths].values
    groups = np.array(df_gasex['Name'])
    scores[target] = []
    i = 0

    for train_ind, test_ind in gkf.split(X = x, y = y, groups = groups):
        #print(df_gasex.loc[test_ind, 'Name'])
        print(target, "CV-{}".format(i))
        rf, mse, wavD, sorted_ind = RF_variable_selection(x=x[train_ind], y= y[train_ind])
        #sns.histplot(x = y[test_ind].flatten(), hue = df_gasex.loc[test_ind, 'Ntreatment'])
        #plt.title(target + '-CV{}'.format(i))
        #plt.show()
        i += 1
        x_test = x[test_ind]
        x_test = x_test[:, sorted_ind]
        x_test = x_test[:, wavD:]
        y_predict = rf.predict(x_test)
        r2 = r2_score(y[test_ind], y_predict)
        print('External Validation Goodness-of-Fit', r2)
        scores[target].append(r2)

df_scores_GasExRFR = pd.DataFrame(scores)
#df_scores_GasExRFR.to_csv('Scores_GasExRFR.csv', index = False)

Index(['E', 'A', 'Ci', 'gsw', 'WUE'], dtype='object')
E CV-0
Optimised MSEP  1.036016376067022e-06
Wavelengths to be discarded: 295
0.6019216182750179
External Validation Goodness-of-Fit 0.027958657754702188
E CV-1
Optimised MSEP  1.0289966248527261e-06
Wavelengths to be discarded: 325
0.5827340217358516
External Validation Goodness-of-Fit -0.3821080675582855
E CV-2
Optimised MSEP  9.810026593532773e-07
Wavelengths to be discarded: 325
0.5592787906133985
External Validation Goodness-of-Fit -0.0824490766925996
E CV-3


KeyboardInterrupt: 

In [4]:
## NUTES RF-R ##

# Same for Nutes
gkf = GroupKFold(n_splits=5)
print(df_nutes.columns[18:30])
targets = list(df_nutes.columns[18:30])

scores = {}
for target in targets:
    y = df_nutes_R2[target].values
    x = df_nutes_R2[wavelengths].values
    groups = np.array(df_nutes_R2['Hybrid'])
    scores[target] = []
    i = 0

    for train_ind, test_ind in gkf.split(X = x, y = y, groups = groups):
        #print(df_gasex.loc[test_ind, 'Name'])
        print(target, "CV-{}".format(i))
        rf, mse, wavD, sorted_ind = RF_variable_selection(x=x[train_ind], y= y[train_ind])
        #sns.histplot(x = y[test_ind].flatten(), hue = df_gasex.loc[test_ind, 'Ntreatment'])
        #plt.title(target + '-CV{}'.format(i))
        #plt.show()
        i += 1
        x_test = x[test_ind]
        x_test = x_test[:, sorted_ind]
        x_test = x_test[:, wavD:]
        y_predict = rf.predict(x_test)
        r2 = r2_score(y[test_ind], y_predict)
        print('External Validation Goodness-of-Fit', r2)
        scores[target].append(r2)

df_scores_NutesRFR = pd.DataFrame(scores)
df_scores_NutesRFR.to_csv('Scores_NutesRFR.csv', index = False)


Index(['N', 'P', 'K', 'Ca', 'Mg', 'Na', 'S', 'Zn', 'Mn', 'Fe', 'Cu', 'B'], dtype='object')
N CV-0
Optimised MSEP  0.11946455522281023
Wavelengths to be discarded: 240
0.9491423870758287
External Validation Goodness-of-Fit 0.5863602407905693
N CV-1
Optimised MSEP  0.14313743183251235
Wavelengths to be discarded: 20
0.9421288978858289
External Validation Goodness-of-Fit 0.5955509191675585
N CV-2
Optimised MSEP  0.12269080729064046
Wavelengths to be discarded: 205
0.9441747013782376
External Validation Goodness-of-Fit 0.6523861679571756
N CV-3
Optimised MSEP  0.13994578960591136
Wavelengths to be discarded: 270
0.9475917718819976
External Validation Goodness-of-Fit 0.5475072013795885
N CV-4
Optimised MSEP  0.12289090572839467
Wavelengths to be discarded: 210
0.9466556398338348
External Validation Goodness-of-Fit 0.5311648964077109
P CV-0
Optimised MSEP  0.001106634376216932
Wavelengths to be discarded: 305
0.9368482004583891
External Validation Goodness-of-Fit 0.5763793416614111
P CV-1
Op

In [14]:
# SVM
from sklearn.svm import SVR

def SVM_R(X, y):
    param_grid = {'kernel': ['linear', 'poly'],
                  'C': [0.1, 0.5, 1, 5, 10]}
    svr = SVR()
    grid_search = GridSearchCV(estimator=svr, param_grid=param_grid, cv = 3)
    grid_search.fit(X, y)
    best_svr = grid_search.best_estimator_
    return best_svr


In [15]:
## GASEX SVR ##

print(df_gasex.columns[3:8])
targets = list(df_gasex.columns[3:8])
gkf = GroupKFold(n_splits=5)
scores = {}
for target in targets:
    y = df_gasex[target].values
    x = df_gasex[wavelengths].values
    groups = np.array(df_gasex['Name'])
    scores[target] = []
    i = 0

    for train_ind, test_ind in gkf.split(X = x, y = y, groups = groups):
        #print(df_gasex.loc[test_ind, 'Name'])
        print(target, "CV-{}".format(i))
        svr = SVM_R(x[train_ind], y[train_ind])
        #sns.histplot(x = y[test_ind].flatten(), hue = df_gasex.loc[test_ind, 'Ntreatment'])
        #plt.title(target + '-CV{}'.format(i))
        #plt.show()
        i += 1
        x_test = x[test_ind]
        y_predict = svr.predict(x_test)
        r2 = r2_score(y[test_ind], y_predict)
        print('External Validation Goodness-of-Fit', r2)
        scores[target].append(r2)

df_scores_GasExSVR = pd.DataFrame(scores)
df_scores_GasExSVR.to_csv('Scores_GasExSVR.csv', index = False)

Index(['E', 'A', 'Ci', 'gsw', 'WUE'], dtype='object')
E CV-0
E-CV0
External Validation Goodness-of-Fit -0.09120555784374185
E CV-1
E-CV1
External Validation Goodness-of-Fit -0.17394720902688143
E CV-2
E-CV2
External Validation Goodness-of-Fit -0.13360718710612973
E CV-3
E-CV3
External Validation Goodness-of-Fit -0.4299017142163648
E CV-4
E-CV4
External Validation Goodness-of-Fit -0.1465565943610656
A CV-0
A-CV0
External Validation Goodness-of-Fit 0.18903480965716923
A CV-1
A-CV1
External Validation Goodness-of-Fit 0.14022099141101718
A CV-2
A-CV2
External Validation Goodness-of-Fit 0.15769344619082826
A CV-3
A-CV3
External Validation Goodness-of-Fit -0.03747270197287311
A CV-4
A-CV4
External Validation Goodness-of-Fit 0.1783224938509096
Ci CV-0
Ci-CV0
External Validation Goodness-of-Fit 0.1053464583692848
Ci CV-1
Ci-CV1
External Validation Goodness-of-Fit -0.1020960532115045
Ci CV-2
Ci-CV2
External Validation Goodness-of-Fit 0.042629700458964304
Ci CV-3
Ci-CV3
External Validation Goodn

In [16]:
## NUTES SVR ##

# Same for Nutes
gkf = GroupKFold(n_splits=5)
print(df_nutes.columns[18:30])
targets = list(df_nutes.columns[18:30])

scores = {}
for target in targets:
    y = df_nutes_R2[target].values
    x = df_nutes_R2[wavelengths].values
    groups = np.array(df_nutes_R2['Hybrid'])
    scores[target] = []
    i = 0

    for train_ind, test_ind in gkf.split(X = x, y = y, groups = groups):
        #print(df_gasex.loc[test_ind, 'Name'])
        print(target, "CV-{}".format(i))
        svr = SVM_R(x[train_ind], y[train_ind])
        print(target + '-CV{}'.format(i))
        #sns.histplot(x = y[test_ind].flatten(), hue = df_gasex.loc[test_ind, 'Ntreatment'])
        #plt.title(target + '-CV{}'.format(i))
        #plt.show()
        i += 1
        x_test = x[test_ind]
        y_predict = svr.predict(x_test)
        r2 = r2_score(y[test_ind], y_predict)
        print('External Validation Goodness-of-Fit', r2)
        scores[target].append(r2)

df_scores_NutesSVR = pd.DataFrame(scores)
df_scores_NutesSVR.to_csv('Scores_NutesSVR.csv', index = False)

Index(['N', 'P', 'K', 'Ca', 'Mg', 'Na', 'S', 'Zn', 'Mn', 'Fe', 'Cu', 'B'], dtype='object')
N CV-0
N-CV0
External Validation Goodness-of-Fit 0.685050019279642
N CV-1
N-CV1
External Validation Goodness-of-Fit 0.6571002435068787
N CV-2
N-CV2
External Validation Goodness-of-Fit 0.7038904352846914
N CV-3
N-CV3
External Validation Goodness-of-Fit 0.5456668920774574
N CV-4
N-CV4
External Validation Goodness-of-Fit 0.7598936882598162
P CV-0
P-CV0
External Validation Goodness-of-Fit -0.0017617825500118034
P CV-1
P-CV1
External Validation Goodness-of-Fit -0.00560482374335014
P CV-2
P-CV2
External Validation Goodness-of-Fit -0.01695792591486489
P CV-3
P-CV3
External Validation Goodness-of-Fit -0.0004761699476019121
P CV-4
P-CV4
External Validation Goodness-of-Fit -0.30998060960543894
K CV-0
K-CV0
External Validation Goodness-of-Fit 0.2773130131712417
K CV-1
K-CV1
External Validation Goodness-of-Fit -0.2964429200309593
K CV-2
K-CV2
External Validation Goodness-of-Fit 0.2307689491506969
K CV-3
K-CV